In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader
import time
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, f1_score, recall_score
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from imblearn.under_sampling import RandomUnderSampler
import itertools

In [ ]:
from dataloader_creator import CreatorDL
creator = CreatorDL(seed=42, bs=2048)

In [ ]:
df_UNSW = creator.reader("NF-UNSW-NB15-v3")

df_train_UNSW, df_test_UNSW, df_val_UNSW = creator.splitter(df_UNSW)

train_loader_UNSW, test_loader_UNSW, val_loader_UNSW = creator.balancer(df_train_UNSW, df_test_UNSW, df_val_UNSW)

In [ ]:
df_BOT= creator.reader("NF-BoT-IoT-v3")

df_train_BOT, df_test_BOT, df_val_BOT = creator.splitter(df_BOT)

train_loader_BOT, test_loader_BOT, val_loader_BOT = creator.balancer(df_train_BOT, df_test_BOT, df_val_BOT)

In [ ]:
df_CIC= creator.reader("NF-CICIDS2018-v3")

df_train_CIC, df_test_CIC, df_val_CIC = creator.splitter(df_CIC)

train_loader_CIC, test_loader_CIC, val_loader_CIC = creator.balancer(df_train_CIC, df_test_CIC, df_val_CIC)

In [ ]:
train_loaders = [train_loader_UNSW, train_loader_BOT, train_loader_CIC]
test_loaders = [test_loader_UNSW, test_loader_BOT, test_loader_CIC]
val_loaders = [val_loader_UNSW, val_loader_BOT, val_loader_CIC]

In [ ]:
INPUT_DIM = 32

class IDSBranchyNet(nn.Module):
    def __init__(self, input_dim=INPUT_DIM, num_classes=2):
        super(IDSBranchyNet, self).__init__()
        
        self.alpha = nn.Parameter(torch.tensor([0.4])) 

        self.shared_layers = nn.Sequential(
            nn.Linear(input_dim, input_dim * 2),
            nn.ReLU(),
        )
        
        self.exit1_layers = nn.Sequential(
            nn.Linear(input_dim * 2, num_classes)
        )
        
        self.exit2_layers = nn.Sequential(
            nn.Linear(input_dim * 2, 1024),
            nn.LeakyReLU(),
            nn.Dropout(0.2),
            nn.Linear(1024, 2048),
            nn.LeakyReLU(),
            nn.Dropout(0.2),
            nn.Linear(2048, 2048),
            nn.LeakyReLU(),
            nn.Dropout(0.2),
            nn.Linear(2048, 1024),
            nn.LeakyReLU(),
            nn.Dropout(0.2),
            nn.Linear(1024, num_classes)
        )

    def get_threshold(self):
        return torch.sigmoid(self.alpha)

    def forward_exit1(self, x):
        features = self.shared_layers(x)
        return self.exit1_layers(features)

    def forward_exit2(self, x):
        features = self.shared_layers(x)
        return self.exit2_layers(features)

model = IDSBranchyNet()

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

In [ ]:
def train_model(model, train_loaders, val_loaders, epochs, lr, device, patience=15):

    optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=1e-4)
    scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.001, patience=7)
    
    model.to(device)
    criterion = nn.CrossEntropyLoss()
    sigmoid = nn.Sigmoid()  

    metrics = [
        'loss1_a', 'loss1_b', 'loss1_c', 'loss_ex1_avg',
        'loss2_a', 'loss2_b', 'loss2_c', 'loss_ex2_avg',
        'l_joint', 'total_loss'
    ]

    history = {
        'train': {k: [] for k in metrics},
        'val': {k: [] for k in metrics}
    }

    best_val_loss = float('inf')
    epochs_no_improve = 0
    best_model_state = None

    max_train_batches = max(len(l) for l in train_loaders) 
    train_iter_loaders = [itertools.cycle(l) if len(l) < max_train_batches else l for l in train_loaders]
    
    max_val_batches = max(len(l) for l in val_loaders)
    val_iter_loaders = [itertools.cycle(l) if len(l) < max_val_batches else l for l in val_loaders]

    for epoch in range(epochs):
        model.train()
        
        running_metrics = {k: 0.0 for k in metrics}
        total_steps = 0

        loader_iterators = [iter(l) for l in train_iter_loaders]
                
        for _ in range(max_train_batches):
            try:
                batches = [next(it) for it in loader_iterators]
            except StopIteration:
                break
            
            optimizer.zero_grad()

            current_threshold = model.get_threshold()

            (inputs_a, labels_a) = batches[0]
            (inputs_b, labels_b) = batches[1]
            (inputs_c, labels_c) = batches[2]
            
            inputs_a, labels_a = inputs_a.to(device), labels_a.to(device)
            inputs_b, labels_b = inputs_b.to(device), labels_b.to(device)
            inputs_c, labels_c = inputs_c.to(device), labels_c.to(device)
            total_samples = inputs_a.size(0) + inputs_b.size(0) + inputs_c.size(0)

            out1_a = model.forward_exit1(inputs_a)
            out1_b = model.forward_exit1(inputs_b)
            out1_c = model.forward_exit1(inputs_c)

            curr_thresh_val = current_threshold.item()

            probs_a = F.softmax(out1_a, dim=1)
            conf_a, _ = torch.max(probs_a, dim=1)

            probs_b = F.softmax(out1_b, dim=1)
            conf_b, _ = torch.max(probs_b, dim=1)

            probs_c = F.softmax(out1_c, dim=1)
            conf_c, _ = torch.max(probs_c, dim=1)

            mask_a_ex1 = conf_a > curr_thresh_val
            mask_b_ex1 = conf_b > curr_thresh_val
            mask_c_ex1 = conf_c > curr_thresh_val

            mask_a_ex2 = conf_a < curr_thresh_val
            mask_b_ex2 = conf_b < curr_thresh_val
            mask_c_ex2 = conf_c < curr_thresh_val

            k = 10

            prob_exit1_a = sigmoid(k * (conf_a - current_threshold))
            prob_exit1_b = sigmoid(k * (conf_b - current_threshold))
            prob_exit1_c = sigmoid(k * (conf_c - current_threshold))
            soft_count_exit1 = torch.sum(prob_exit1_a) + torch.sum(prob_exit1_b) + torch.sum(prob_exit1_c)
        
            prob_exit2_a = sigmoid(k * (current_threshold - conf_a))
            prob_exit2_b = sigmoid(k * (current_threshold - conf_b))
            prob_exit2_c = sigmoid(k * (current_threshold - conf_c))
            soft_count_exit2 = torch.sum(prob_exit2_a) + torch.sum(prob_exit2_b) + torch.sum(prob_exit2_c)

            loss1_a = torch.tensor(0.0, device=device)
            loss1_b = torch.tensor(0.0, device=device)
            loss1_c = torch.tensor(0.0, device=device)

            if mask_a_ex1.any():
                loss1_a = criterion(out1_a[mask_a_ex1], labels_a[mask_a_ex1])
            if mask_b_ex1.any():
                loss1_b = criterion(out1_b[mask_b_ex1], labels_b[mask_b_ex1])
            if mask_c_ex1.any():
                loss1_c = criterion(out1_c[mask_c_ex1], labels_c[mask_c_ex1])

            # loss1_a = criterion(out1_a, labels_a)
            # loss1_b = criterion(out1_b, labels_b)
            # loss1_c = criterion(out1_c, labels_c)
                
            loss_ex1_avg = (loss1_a + loss1_b + loss1_c) / 3

            loss2_a = torch.tensor(0.0, device=device)
            loss2_b = torch.tensor(0.0, device=device)
            loss2_c = torch.tensor(0.0, device=device)

            if mask_a_ex2.any():
                loss2_a = criterion(model.forward_exit2(inputs_a[mask_a_ex2]), labels_a[mask_a_ex2])
            if mask_b_ex2.any():
                loss2_b = criterion(model.forward_exit2(inputs_b[mask_b_ex2]), labels_b[mask_b_ex2])
            if mask_c_ex2.any():
                loss2_c = criterion(model.forward_exit2(inputs_c[mask_c_ex2]), labels_c[mask_c_ex2])

            # loss2_a = criterion(model.forward_exit2(inputs_a), labels_a)
            # loss2_b = criterion(model.forward_exit2(inputs_b), labels_b)
            # loss2_c = criterion(model.forward_exit2(inputs_c), labels_c)

            loss_ex2_avg = (loss2_a + loss2_b + loss2_c) / 3

            l_joint = loss_ex1_avg + loss_ex2_avg
                        
            total_loss = l_joint + (soft_count_exit2 / total_samples)

            total_loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()

            running_metrics['loss1_a'] += loss1_a.item()
            running_metrics['loss1_b'] += loss1_b.item()
            running_metrics['loss1_c'] += loss1_c.item()
            running_metrics['loss_ex1_avg'] += loss_ex1_avg.item()
            
            running_metrics['loss2_a'] += loss2_a.item()
            running_metrics['loss2_b'] += loss2_b.item()
            running_metrics['loss2_c'] += loss2_c.item()
            running_metrics['loss_ex2_avg'] += loss_ex2_avg.item()
            
            running_metrics['l_joint'] += l_joint.item()
            running_metrics['total_loss'] += total_loss.item()
            
            total_steps += 1

        for k in metrics:
            history['train'][k].append(running_metrics[k] / total_steps)
        
        epoch_train_loss = history['train']['total_loss'][-1]

        model.eval()
        running_metrics_val = {k: 0.0 for k in metrics}
        total_steps_val = 0
        
        with torch.no_grad():
            current_threshold = model.get_threshold()
            curr_thresh_val = current_threshold.item()

            for _ in range(max_val_batches):
                try:
                    batches = [next(it) for it in val_loader_iterators]
                except StopIteration:
                    break

                (inputs_a, labels_a) = batches[0]
                (inputs_b, labels_b) = batches[1]
                (inputs_c, labels_c) = batches[2]
                    
                inputs_a, labels_a = inputs_a.to(device), labels_a.to(device)
                inputs_b, labels_b = inputs_b.to(device), labels_b.to(device)
                inputs_c, labels_c = inputs_c.to(device), labels_c.to(device)
                total_samples = inputs_a.size(0) + inputs_b.size(0) + inputs_c.size(0)
    
                out1_a = model.forward_exit1(inputs_a)
                out1_b = model.forward_exit1(inputs_b)
                out1_c = model.forward_exit1(inputs_c)
    
                curr_thresh_val = current_threshold.item()

                probs_a = F.softmax(out1_a, dim=1)
                conf_a, _ = torch.max(probs_a, dim=1)
    
                probs_b = F.softmax(out1_b, dim=1)
                conf_b, _ = torch.max(probs_b, dim=1)
    
                probs_c = F.softmax(out1_c, dim=1)
                conf_c, _ = torch.max(probs_c, dim=1)
    
                mask_a_ex1 = conf_a > curr_thresh_val
                mask_b_ex1 = conf_b > curr_thresh_val
                mask_c_ex1 = conf_c > curr_thresh_val
    
                mask_a_ex2 = conf_a < curr_thresh_val
                mask_b_ex2 = conf_b < curr_thresh_val
                mask_c_ex2 = conf_c < curr_thresh_val
    
                loss1_a = torch.tensor(0.0, device=device)
                loss1_b = torch.tensor(0.0, device=device)
                loss1_c = torch.tensor(0.0, device=device)
    
                if mask_a_ex1.any():
                    loss1_a = criterion(out1_a[mask_a_ex1], labels_a[mask_a_ex1])
                
                if mask_b_ex1.any():
                    loss1_b = criterion(out1_b[mask_b_ex1], labels_b[mask_b_ex1])
    
                if mask_c_ex1.any():
                    loss1_c = criterion(out1_c[mask_c_ex1], labels_c[mask_c_ex1])

                # loss1_a = criterion(out1_a, labels_a)
                # loss1_b = criterion(out1_b, labels_b)
                # loss1_c = criterion(out1_c, labels_c)
    
                loss_ex1_avg = (loss1_a + loss1_b + loss1_c) / 3
                
                real_count_exit2 = mask_a_ex2.sum() + mask_b_ex2.sum() + mask_c_ex2.sum()
    
                loss2_a = torch.tensor(0.0, device=device)
                loss2_b = torch.tensor(0.0, device=device)
                loss2_c = torch.tensor(0.0, device=device)
    
                if mask_a_ex2.any(): loss2_a = criterion(model.forward_exit2(inputs_a[mask_a_ex2]), labels_a[mask_a_ex2])
                if mask_b_ex2.any(): loss2_b = criterion(model.forward_exit2(inputs_b[mask_b_ex2]), labels_b[mask_b_ex2])
                if mask_c_ex2.any(): loss2_c = criterion(model.forward_exit2(inputs_c[mask_c_ex2]), labels_c[mask_c_ex2])
    
                # loss2_a = criterion(model.forward_exit2(inputs_a), labels_a)
                # loss2_b = criterion(model.forward_exit2(inputs_b), labels_b)
                # loss2_c = criterion(model.forward_exit2(inputs_c), labels_c)
    
                loss_ex2_avg = (loss2_a + loss2_b + loss2_c) / 3
    
                l_joint = loss_ex1_avg + loss_ex2_avg
                                                    
                total_loss = l_joint + (real_count_exit2 / total_samples)
                
                running_metrics_val['loss1_a'] += loss1_a.item()
                running_metrics_val['loss1_b'] += loss1_b.item()
                running_metrics_val['loss1_c'] += loss1_c.item()
                running_metrics_val['loss_ex1_avg'] += loss_ex1_avg.item()
                
                running_metrics_val['loss2_a'] += loss2_a.item()
                running_metrics_val['loss2_b'] += loss2_b.item()
                running_metrics_val['loss2_c'] += loss2_c.item()
                running_metrics_val['loss_ex2_avg'] += loss_ex2_avg.item()
                
                running_metrics_val['l_joint'] += l_joint.item()
                running_metrics_val['total_loss'] += total_loss.item()
                
                total_steps_val += 1

        for k in metrics:
            history['val'][k].append(running_metrics_val[k] / total_steps_val)

        epoch_val_loss = history['val']['total_loss'][-1]

        print(f'Epoch [{epoch+1}/{epochs}] | Train Total: {epoch_train_loss:.4f} | Val Total: {epoch_val_loss:.4f} | Alpha: {curr_thresh_val}')
        
        if epoch_val_loss < best_val_loss:
            best_val_loss = epoch_val_loss
            epochs_no_improve = 0
            best_model_state = model.state_dict()
        else:
            epochs_no_improve += 1
            if epochs_no_improve >= patience:
                print("Early stopping...")
                if best_model_state: model.load_state_dict(best_model_state)
                break
                
        scheduler.step(epoch_val_loss) 
                
    epochs_range = range(1, len(history['train']['total_loss']) + 1)
    
    fig, axs = plt.subplots(1, 3, figsize=(20, 6))
    
    ax = axs[0]
    ax.set_title("Exit 1")
    ax.plot(epochs_range, history['train']['loss1_a'], label='Tr A', alpha=0.6)
    ax.plot(epochs_range, history['train']['loss1_b'], label='Tr B', alpha=0.6)
    ax.plot(epochs_range, history['train']['loss1_c'], label='Tr C', alpha=0.6)
    ax.plot(epochs_range, history['train']['loss_ex1_avg'], label='Tr Avg', linewidth=2)
    
    ax.plot(epochs_range, history['val']['loss1_a'], label='Val A', color='black', linestyle='--')
    ax.plot(epochs_range, history['val']['loss1_b'], label='Val B', color='black', linestyle='--')
    ax.plot(epochs_range, history['val']['loss1_c'], label='Val C', color='black', linestyle='--')
    ax.plot(epochs_range, history['val']['loss_ex1_avg'], label='Val Avg', color='black', linestyle='--', linewidth=2)
    ax.set_xlabel('Epochs')
    ax.set_ylabel('Loss')
    ax.legend()
    ax.grid(True)

    ax = axs[1]
    ax.set_title("Exit 2")
    ax.plot(epochs_range, history['train']['loss2_a'], label='Tr A', color='blue', alpha=0.6)
    ax.plot(epochs_range, history['train']['loss2_b'], label='Tr B', color='green', alpha=0.6)
    ax.plot(epochs_range, history['train']['loss2_c'], label='Tr C', color='red', alpha=0.6)
    ax.plot(epochs_range, history['train']['loss_ex2_avg'], label='Tr Avg', color='black', linewidth=2)
    
    ax.plot(epochs_range, history['val']['loss2_a'], label='Val A', color='black', linestyle='--')
    ax.plot(epochs_range, history['val']['loss2_b'], label='Val B', color='black', linestyle='--')
    ax.plot(epochs_range, history['val']['loss2_c'], label='Val C', color='black', linestyle='--')
    ax.plot(epochs_range, history['val']['loss_ex2_avg'], label='Val Avg', color='black', linestyle='--', linewidth=2)
    ax.set_xlabel('Epochs')
    ax.legend()
    ax.grid(True)

    ax = axs[2]
    ax.set_title("Global Optimization")
    ax.plot(epochs_range, history['train']['l_joint'], label='Tr Joint (Ex1 + Ex2)', color='purple')
    ax.plot(epochs_range, history['train']['total_loss'], label='Tr Total (Joint + (NEx2 / N))', color='orange', linewidth=2)
    
    ax.plot(epochs_range, history['val']['l_joint'], label='Val Joint', color='purple', linestyle='--')
    ax.plot(epochs_range, history['val']['total_loss'], label='Val Total', color='orange', linestyle='--', linewidth=2)
    ax.set_xlabel('Epochs')
    ax.legend()
    ax.grid(True)

    plt.tight_layout()
    plt.show()
    
    return curr_thresh_val

In [ ]:
def evaluate_model(model, loader, confidence_threshold, device):
    model.to(device)
    model.eval()

    total_samples = len(loader.dataset)
    all_predictions = []
    all_labels = []
    exited_early_count = 0
    total_inference_time = 0

    with torch.no_grad():
        for samples, labels in loader:
            samples, labels = samples.to(device), labels.to(device)

            start_time = time.perf_counter()
            
            branch_output = model.forward_exit1(samples)
            
            branch_prob = F.softmax(branch_output, dim=1)
            trusts, branch_preds = torch.max(branch_prob, 1)

            batch_predictions = torch.zeros_like(labels)
            
            early_exit_mask = trusts > confidence_threshold
            
            if early_exit_mask.any():
                batch_predictions[early_exit_mask] = branch_preds[early_exit_mask]
                exited_early_count += early_exit_mask.sum().item()

            main_branch_mask = ~early_exit_mask
            if main_branch_mask.any():
                
                samples_to_main = samples[main_branch_mask]
                
                main_output = model.forward_exit2(samples_to_main)
                
                main_prob = F.softmax(main_output, dim=1)
                _, main_preds = torch.max(main_prob, 1)
                
                batch_predictions[main_branch_mask] = main_preds

            end_time = time.perf_counter()
            total_inference_time += (end_time - start_time)

            all_predictions.append(batch_predictions.cpu())
            all_labels.append(labels.cpu())

    final_predictions = torch.cat(all_predictions)
    y_data = torch.cat(all_labels)

    correct = (final_predictions == y_data).sum().item()
    accuracy = 100 * correct / total_samples
    exit_rate = 100 * exited_early_count / total_samples
    avg_time_ms = (total_inference_time / total_samples) * 1000

    cm = confusion_matrix(y_data.numpy(), final_predictions.numpy())

    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=['Normal', 'Ataque'],
                yticklabels=['Normal', 'Ataque'])
    plt.xlabel('Rótulo Previsto')
    plt.ylabel('Rótulo Verdadeiro')
    plt.title(f'Matriz de Confusão (Limiar de Confiança = {confidence_threshold})')
    plt.show()

    tn, fp, fn, tp = cm.ravel()
    
    f1 = f1_score(y_data.numpy(), final_predictions.numpy())
    
    tpr = recall_score(y_data.numpy(), final_predictions.numpy())

    tnr = tn / (tn + fp) if (tn + fp) > 0 else 0
    
    print(f"True Positives (TP): {tp}")
    print(f"True Negatives (TN): {tn}")
    print(f"False Positives (FP): {fp}")
    print(f"False Negatives (FN): {fn}\n")
    
    print(f"F1 Score: {f1:.4f}")
    print(f"True Positive Rate (TPR) / Recall: {tpr:.4f}")
    print(f"True Negative Rate (TNR) / Specificity: {tnr:.4f}")

    return {
            'accuracy': accuracy,
            'exit_rate': exit_rate,
            'avg_inference_time_ms': avg_time_ms,
            'exited_early_count': exited_early_count,
            'total_samples': total_samples,
            'f1': f1
        }

In [ ]:
modelname = 'teste_ljoint6'
modelname

In [ ]:
epochs = 10

limiar = train_model(
    model, 
    train_loaders, 
    val_loaders, 
    epochs, 
    lr=0.01,
    device=device
)
torch.save(model.state_dict(), f'models/{modelname}.pth')
print(f"\nModelo treinado e salvo em 'models/{modelname}.pth'")

In [ ]:
model.load_state_dict(torch.load(f'models/{modelname}.pth'))
print(f"Modelo 'models/{modelname}.pth' carregado\n")

print(f"Base: UNSW")
results = evaluate_model(model, test_loaders[0], limiar, device=device)
print("-" * 20)
print(f"  Accuracy: {results['accuracy']:.4f}%")
print(f"  Avg. Inference Time: {results['avg_inference_time_ms']:.4f} ms")
print(f"  Early Exit Rate: {results['exit_rate']:.4f}% ({results['exited_early_count']}/{results['total_samples']})")
print("-" * 20)

print(f"\nBase: BOT")
results = evaluate_model(model, test_loaders[1], limiar, device=device)
print("-" * 20)
print(f"  Accuracy: {results['accuracy']:.4f}%")
print(f"  Avg. Inference Time: {results['avg_inference_time_ms']:.4f} ms")
print(f"  Early Exit Rate: {results['exit_rate']:.4f}% ({results['exited_early_count']}/{results['total_samples']})")
print("-" * 20)

print(f"\nBase: CIC")
results = evaluate_model(model, test_loaders[2], limiar, device=device)
print("-" * 20)
print(f"  Accuracy: {results['accuracy']:.4f}%")
print(f"  Avg. Inference Time: {results['avg_inference_time_ms']:.4f} ms")
print(f"  Early Exit Rate: {results['exit_rate']:.4f}% ({results['exited_early_count']}/{results['total_samples']})")
print("-" * 20)